In [1]:
!gpustat

seven                Wed Mar 10 18:02:48 2021  440.64
[0] TITAN RTX        | 34'C,   0 % |    11 / 24220 MB |
[1] TITAN RTX        | 71'C,  37 % |  2662 / 24220 MB | cwb(2651M)
[2] TITAN RTX        | 32'C,   0 % |    11 / 24220 MB |
[3] TITAN RTX        | 86'C,  98 % |  9928 / 24220 MB | cwb(3411M) cwb(3063M) cwb(3443M)


In [2]:
from os.path import join as pathjoin
from data_processing import *
from interpretation import *
from models import *
from training import *

In [3]:
ddd = pd.read_csv('/home/mlepekhin/data/ru_train')
ddd = ddd.append(pd.read_csv('/home/mlepekhin/data/en_train'))
ddd = ddd.append(pd.read_csv('/home/mlepekhin/data/new_ru_attacked_50.csv'))
ddd = ddd.append(pd.read_csv('/home/mlepekhin/data/en_attacked_50.csv')).sample(frac=1, random_state=42)
print(ddd.shape)
ddd.head()

(3909, 7)


,Unnamed: 0,target,text,changed_words_num,new_model_target,old_model_target,old_text
82,82,A12,"Теги : Cubitek Тайваньская биржа Cubitek , до ...",19.0,A7,A12,"Теги : Cubitek Тайваньская компания Cubitek , ..."
92,92,A4,"﻿ II "" А что , если уж и был обыск ? Что , дум...",11.0,A11,A4,"﻿ II "" А что , если уж и был обыск ? Что , есл..."
771,902,A4,Kingdom Hearts - Two Destinies are One Chapter...,NaN,NaN,NaN,NaN
400,400,A8,"Берзиньш душевно согласился него себя "" Такую...",27.0,A17,A8,"Берзиньш душевно принял у себя "" Новую волну ..."
450,450,A17,"Ориентироваться платные фильм "" Техасская ре...",37.0,A12,A17,"Смотреть онлайн фильм "" Техасская резня бенз..."


In [4]:
ddd.to_csv('enru_attacked_train_50.csv')

Let's download all the data.

In [6]:
DATA_DIR = '/home/mlepekhin/data'
MODELS_DIR = '/home/mlepekhin/models'
MODEL_ID = 'rubert_enru_attacked_50' 
CHECKPOINTS_DIR = pathjoin(MODELS_DIR, MODEL_ID, 'checkpoints')
BEST_MODEL = pathjoin(CHECKPOINTS_DIR, 'best.th')

In [7]:
transformer_model = 'DeepPavlov/rubert-base-cased'
MAX_TOKENS = 512

In [8]:
tokenizer = PretrainedTransformerTokenizer(transformer_model)

03112021 09:48:20|INFO|transformers.configuration_utils| loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/DeepPavlov/rubert-base-cased/config.json from cache at /home/mlepekhin/.cache/torch/transformers/b1ea51df212fa070e3a8a7d04dedb4800621e5a5bf504ecdb305faa622b46aa0.41d1cb30da8abef9028a44a17bd9c152daca0bd46e409bc271f324a28d109450
03112021 09:48:20|INFO|transformers.configuration_utils| Model config BertConfig {
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "po

In [9]:
train_dataset_reader = build_transformer_dataset_reader(transformer_model, lower=True)
val_dataset_reader = build_transformer_dataset_reader(transformer_model, lower=True)

train_data, dev_data = read_data(
    "enru_attacked_train_50.csv", 
    pathjoin(DATA_DIR, "manual_genres", "all.csv"),
    train_dataset_reader, 
    val_dataset_reader
)

vocab = build_vocab(train_data + dev_data)

train_data.index_with(vocab)
dev_data.index_with(vocab)

03112021 09:48:26|INFO|transformers.configuration_utils| loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/DeepPavlov/rubert-base-cased/config.json from cache at /home/mlepekhin/.cache/torch/transformers/b1ea51df212fa070e3a8a7d04dedb4800621e5a5bf504ecdb305faa622b46aa0.41d1cb30da8abef9028a44a17bd9c152daca0bd46e409bc271f324a28d109450
03112021 09:48:26|INFO|transformers.configuration_utils| Model config BertConfig {
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "po

03112021 09:48:33|INFO|transformers.tokenization_utils| loading file https://s3.amazonaws.com/models.huggingface.co/bert/DeepPavlov/rubert-base-cased/vocab.txt from cache at /home/mlepekhin/.cache/torch/transformers/3e164bb7396e401202e250721569fb407583681bb6ea0c34f431af622435a3d8.67d40fedd426f94e9ea8d75f879bbc353613af6b908324e8eca582d4cfa9b0eb
03112021 09:48:33|INFO|transformers.tokenization_utils| loading file https://s3.amazonaws.com/models.huggingface.co/bert/DeepPavlov/rubert-base-cased/added_tokens.json from cache at None
03112021 09:48:33|INFO|transformers.tokenization_utils| loading file https://s3.amazonaws.com/models.huggingface.co/bert/DeepPavlov/rubert-base-cased/special_tokens_map.json from cache at /home/mlepekhin/.cache/torch/transformers/d44047cea679f35c9ce4f09172821fc62d3b493f77e63d3aeffd42f51df90ce9.275045728fbf41c11d3dae08b8742c054377e18d92cc7b72b6351152a99b64e4
03112021 09:48:33|INFO|transformers.tokenization_utils| loading file https://s3.amazonaws.com/models.huggin

03112021 09:48:40|INFO|transformers.tokenization_utils| Model name 'DeepPavlov/rubert-base-cased' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, TurkuNLP/bert-base-finnish-cased-v1, TurkuNLP/bert-base-finnish-uncased-v1, wietsedv/bert-base-dutch-cased). Assuming 'DeepPavlov/rubert-base-cased' is a path, a model identifier, or url to a directory containing tokenizer files.
03112021 09:48:42|INFO|transformers.tokenization_utils| loading file https://s3.amazonaws.com/models.huggingface.co/bert/DeepPavlov/rubert-base-cased/vocab.txt from cache at /h

Reading data
<class 'data_processing.ClassificationDatasetReader'> enru_attacked_train_50.csv


03112021 09:50:39|INFO|allennlp.data.vocabulary| Fitting token dictionary from dataset.



Building the vocabulary


In [10]:
model = build_transformer_model(vocab, transformer_model)

Building the model


03112021 09:50:39|INFO|transformers.configuration_utils| loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/DeepPavlov/rubert-base-cased/config.json from cache at /home/mlepekhin/.cache/torch/transformers/b1ea51df212fa070e3a8a7d04dedb4800621e5a5bf504ecdb305faa622b46aa0.41d1cb30da8abef9028a44a17bd9c152daca0bd46e409bc271f324a28d109450
03112021 09:50:39|INFO|transformers.configuration_utils| Model config BertConfig {
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "po

03112021 09:50:49|INFO|transformers.modeling_utils| loading weights file https://cdn.huggingface.co/DeepPavlov/rubert-base-cased/pytorch_model.bin from cache at /home/mlepekhin/.cache/torch/transformers/611b8e3ce80c0751b16e01b0e7c133fffc388089408bb9ef7f8a2d60c9758fd3.71d8ad10edfcd7c68264ea03bc1b14e1f7b9c67affdfe8d6f96e1a6ce2c136ee


In [11]:
if torch.cuda.is_available():
    cuda_device = 0
    model = model.cuda(cuda_device)
else:
    cuda_device = -1
print(cuda_device)

0


In [12]:
!rm -rf {CHECKPOINTS_DIR}
!mkdir -p {CHECKPOINTS_DIR}

In [13]:
train_loader, dev_loader = build_data_loaders(train_data, dev_data)

# You obviously won't want to create a temporary file for your training
# results, but for execution in binder for this course, we need to do this.

trainer = build_classifier_trainer(
    model,
    pathjoin(MODELS_DIR, MODEL_ID, 'checkpoints'),
    train_loader,
    dev_loader,
    10,
    cuda_device=cuda_device
)
print("Starting training")
trainer.train()
print("Finished training")

03112021 09:50:55|INFO|allennlp.training.optimizers| Number of trainable parameters: 178452491
03112021 09:50:55|WARNING|allennlp.training.trainer| You provided a validation dataset but patience was set to None, meaning that early stopping is disabled
03112021 09:50:55|INFO|allennlp.training.trainer| Beginning training.
03112021 09:50:55|INFO|allennlp.training.trainer| Epoch 0/9
03112021 09:50:55|INFO|allennlp.training.trainer| Worker 0 memory usage MB: 5055.228


Starting training


03112021 09:50:56|INFO|allennlp.training.trainer| GPU 0 memory usage MB: 1608
03112021 09:50:56|INFO|allennlp.training.trainer| GPU 1 memory usage MB: 2662
03112021 09:50:56|INFO|allennlp.training.trainer| GPU 2 memory usage MB: 11
03112021 09:50:56|INFO|allennlp.training.trainer| GPU 3 memory usage MB: 9928
03112021 09:50:56|INFO|allennlp.training.trainer| Training


03112021 09:53:07|INFO|allennlp.training.trainer| Validating


03112021 09:53:50|INFO|allennlp.training.tensorboard_writer|                        Training |  Validation
03112021 09:53:50|INFO|allennlp.training.tensorboard_writer| accuracy           |     0.581  |     0.693
03112021 09:53:50|INFO|allennlp.training.tensorboard_writer| gpu_0_memory_MB    |  1608.000  |       N/A
03112021 09:53:50|INFO|allennlp.training.tensorboard_writer| gpu_1_memory_MB    |  2662.000  |       N/A
03112021 09:53:50|INFO|allennlp.training.tensorboard_writer| gpu_2_memory_MB    |    11.000  |       N/A
03112021 09:53:50|INFO|allennlp.training.tensorboard_writer| gpu_3_memory_MB    |  9928.000  |       N/A
03112021 09:53:50|INFO|allennlp.training.tensorboard_writer| loss               |     1.319  |     0.974
03112021 09:53:50|INFO|allennlp.training.tensorboard_writer| reg_loss           |     0.000  |     0.000
03112021 09:53:50|INFO|allennlp.training.tensorboard_writer| worker_0_memory_MB |  5055.228  |       N/A


...

03112021 10:19:40|INFO|allennlp.training.trainer| Epoch duration: 0:03:10.320323
03112021 10:19:40|INFO|allennlp.training.trainer| Estimated training time remaining: 0:03:11
03112021 10:19:40|INFO|allennlp.training.trainer| Epoch 9/9
03112021 10:19:40|INFO|allennlp.training.trainer| Worker 0 memory usage MB: 5055.228
03112021 10:19:40|INFO|allennlp.training.trainer| GPU 0 memory usage MB: 16710
03112021 10:19:40|INFO|allennlp.training.trainer| GPU 1 memory usage MB: 2662
03112021 10:19:40|INFO|allennlp.training.trainer| GPU 2 memory usage MB: 16710
03112021 10:19:40|INFO|allennlp.training.trainer| GPU 3 memory usage MB: 9928
03112021 10:19:40|INFO|allennlp.training.trainer| Training


03112021 10:22:03|INFO|allennlp.training.trainer| Validating


03112021 10:22:48|INFO|allennlp.training.tensorboard_writer|                        Training |  Validation
03112021 10:22:48|INFO|allennlp.training.tensorboard_writer| accuracy           |     0.980  |     0.704
03112021 10:22:48|INFO|allennlp.training.tensorboard_writer| gpu_0_memory_MB    |  16710.000  |       N/A
03112021 10:22:48|INFO|allennlp.training.tensorboard_writer| gpu_1_memory_MB    |  2662.000  |       N/A
03112021 10:22:48|INFO|allennlp.training.tensorboard_writer| gpu_2_memory_MB    |  16710.000  |       N/A
03112021 10:22:48|INFO|allennlp.training.tensorboard_writer| gpu_3_memory_MB    |  9928.000  |       N/A
03112021 10:22:48|INFO|allennlp.training.tensorboard_writer| loss               |     0.062  |     1.480
03112021 10:22:48|INFO|allennlp.training.tensorboard_writer| reg_loss           |     0.000  |     0.000
03112021 10:22:48|INFO|allennlp.training.tensorboard_writer| worker_0_memory_MB |  5055.228  |       N/A


03112021 10:22:50|INFO|allennlp.training.trainer| Epoch duration: 0:03:10.301335
03112021 10:22:50|INFO|allennlp.training.checkpointer| loading best weights


Finished training


In [14]:
!rm "{CHECKPOINTS_DIR}"/*.json "{CHECKPOINTS_DIR}"/model* "{CHECKPOINTS_DIR}"/training*
model.vocab.save_to_files(pathjoin(MODELS_DIR, MODEL_ID, 'vocab'))
!ls -lh "{CHECKPOINTS_DIR}"

03112021 10:22:51|INFO|filelock| Lock 139873155287168 acquired on /home/mlepekhin/models/rubert_enru_attacked_50/vocab/.lock
03112021 10:22:52|INFO|filelock| Lock 139873155287168 released on /home/mlepekhin/models/rubert_enru_attacked_50/vocab/.lock


total 681M
-rw-rw-r-- 1 mlepekhin mlepekhin 681M мар 11 10:06 best.th
drwxrwxr-x 4 mlepekhin mlepekhin 4,0K мар 11 09:50 log
